In [3]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

from enum import IntEnum, IntFlag
from math import ceil, floor
import backtrader as bt
import datetime as dt
import pandas as pd
import inspect
import math
import pdb

In [ ]:
#%%pixie_debugger


def dump_args(func):
    '''Decorator to print function call details - parameters names and effective values.
    
    https://stackoverflow.com/questions/6200270/decorator-to-print-function-call-details-parameters-names-and-effective-values
    '''
    def wrapper(*args, **kwargs):
        func_args = inspect.signature(func).bind(*args, **kwargs).arguments
        func_args_str =  ', '.join('{} = {!r}'.format(*item) for item in func_args.items())
        print(f'{func.__module__}.{func.__qualname__} ( {func_args_str} )')
        return func(*args, **kwargs)
    return wrapper
        
        
class OrderObserver(bt.observer.Observer):
    lines = ('buy', 'sell')

    plotinfo = dict(plot=True, subplot=False, plotlinelabels=True)

    plotlines = dict(
        buy=dict(marker='*', markersize=4.0, color='lime', fillstyle='full'),
        sell=dict(marker='*', markersize=4.0, color='red')
    )

    def next(self):
        for order in self._owner._orderspending:
            if order.data not in self.datas:
                continue
                
            if order.status in [bt.Order.Accepted, bt.Order.Submitted] and order.exectype==bt.Order.StopLimit:
                if order.isbuy():
                    self.lines.buy[0] = order.created.price
                else:
                    self.lines.sell[0] = order.created.price

class CrossoverEvent(IntEnum):
    NONE = 0
    CROSS_UP = 1
    CROSS_DOWN = 2
    ABOVE = 3
    BELOW = 4
    
    
class Patterns(IntFlag):
    NONE = 0,
    Buy = 1,
    Sell = 2,
    Pattern1 = 4,
    Pattern2 = 8,
    Pattern3 = 16,
    Pattern4 = 32,
    Pattern5 = 64,
    Pattern6 = 128,
    Pattern7 = 256,
    Pattern8 = 512

    
class St(bt.Strategy):
    params = (
        ('semaperiod', 8),
        ('lemaperiod', 15),
        ('risk_percent', 1.0),  
        ('pip_size', 0.0001),  
        ('bar_size', dt.timedelta(minutes=240)),
        ('Filter1Enabled', True),
        ('Filter2Enabled', True),
        ('Filter3Enabled', True),
        ('Filter4Enabled', True),
        ('Filter5Enabled', True),
        ('Filter6Enabled', True),
        ('Filter7Enabled', True),
        ('Filter8Enabled', True)
    )
    
    def __init__(self):
        self.last_event = CrossoverEvent.NONE
        self.sema = bt.indicators.ExponentialMovingAverage(self.datas[0], period=self.params.semaperiod)
        self.lema = bt.indicators.ExponentialMovingAverage(self.datas[0], period=self.params.lemaperiod)
        self.open = self.datas[0].open
        self.high = self.datas[0].high
        self.low = self.datas[0].low
        self.close = self.datas[0].close
        self.trade_id = 1
        
    def logdata(self):
        txt = []
        txt.append('{}'.format(len(self)))
           
        txt.append('{}'.format(
            self.data.datetime.datetime(0).isoformat())
        )
        txt.append('{:.5f}'.format(self.data.open[0]))
        txt.append('{:.5f}'.format(self.data.high[0]))
        txt.append('{:.5f}'.format(self.data.low[0]))
        txt.append('{:.5f}'.format(self.data.close[0]))
        print(','.join(txt))
        
    def calc(self):
        time = self.data.datetime.datetime(0)
        shortEma = self.sema[0]
        longEma = self.lema[0]
        open_ = self.open[0]
        low = self.low[0]
        high = self.high[0]
        close = self.close[0]
        buyStrings = []
        sellStrings = []
        
        if shortEma > longEma:
            if self.last_event == CrossoverEvent.ABOVE:
                if high < longEma or high >= shortEma and low <= shortEma or high >= longEma and low <= longEma:
                    self.OnBuyStage3()
            elif self.last_event != CrossoverEvent.CROSS_UP:                    
                self.last_event = CrossoverEvent.CROSS_UP
                
        if shortEma < longEma:
            if self.last_event == CrossoverEvent.BELOW:
                if low > longEma or high >= shortEma and low <= shortEma or high >= longEma and low <= longEma:
                    self.OnSellStage3()
            elif self.last_event != CrossoverEvent.CROSS_DOWN:
                self.last_event = CrossoverEvent.CROSS_DOWN      
                
        if low > shortEma and self.last_event == CrossoverEvent.CROSS_UP:
            self.last_event = CrossoverEvent.ABOVE
        
        if high < shortEma and self.last_event == CrossoverEvent.CROSS_DOWN:
            self.last_event = CrossoverEvent.BELOW
        
    @dump_args
    def OnBuyStage3(self, index:int = 0):
        p = Patterns.NONE;

        if self.params.Filter1Enabled and self.IsLowerHigh(index) and self.IsLowerLow(index):
            p |= Patterns.Pattern1
        
        if self.params.Filter5Enabled and self.IsInsidePeriod(index):
            p |= Patterns.Pattern5
        
        if p != Patterns.NONE:
            self.log_output(index, p, Patterns.Buy)
            entry = self.round_up(self.high[index] + self.params.pip_size)
            limit = entry + self.params.pip_size
            isl = self.round_down(self.low[index] - self.params.pip_size)
            diff = limit - isl
            tp = limit + diff
            pips = diff / self.params.pip_size
            equity = self.broker.get_value()
            s = self.round_down(equity * self.params.risk_percent / 
                                (100 * pips * self.params.pip_size), 1000)
            # print('equity', equity, 'sell', s)
            entry_ord = self.buy(price=entry, plimit=limit, size=s, 
                                 exectype=bt.Order.StopLimit,
                                 # exectype=bt.Order.Limit,  
                                 valid=self.params.bar_size,
                                 tradeid=self.trade_id,
                                 transmit=False)
            isl_ord = self.sell(size=s, 
                                price=isl, 
                                exectype=bt.Order.StopTrail, # exectype=bt.Order.Stop, 
                                trailamount=diff,
                                parent=entry_ord,
                                tradeid=self.trade_id,
                                transmit=False)
            tp_ord = self.sell(size=s, 
                               price=tp, 
                               exectype=bt.Order.Limit, 
                               parent=entry_ord,
                               tradeid=self.trade_id,
                               transmit=True)
            self.trade_id += 1
        
    # @dump_args
    def OnSellStage3(self, index:int = 0):
        pass

    def round_down(self, value, rounding=None):
        rounding = rounding or self.params.pip_size
        return floor(value / rounding) * rounding
            
    def round_up(self, value, rounding=None):
        rounding = rounding or self.params.pip_size
        return ceil(value / rounding) * rounding            
            
    def log_output(self, index:int, p:Patterns, side:Patterns):
        matches = p | side
        time = self.data.datetime.datetime(0)
        open_ = self.open[0]
        low = self.low[0]
        high = self.high[0]
        close = self.close[0]
        print(f'{time}: ({open_:.5f}, {high:.5f}, {low:.5f}, {close:.5f}) {repr(matches)}')
            
    def IsHigherHigh(self, index:int = 0):
        return self.high[index] > self.high[index - 1]

    def IsLowerHigh(self, index:int = 0):      
        return self.high[index] < self.high[index - 1] 

    def IsLowerOrEqualHigh(self, index:int = 0):      
        return self.high[index] <= self.high[index - 1]         

    def IsHigherLow(self, index:int = 0):        
        return self.low[index] > self.low[index - 1]       

    def IsHigherOrEqualLow(self, index:int = 0):        
        return self.low[index] >= self.low[index - 1]

    def IsLowerLow(self, index:int = 0):        
        return self.low[index] < self.low[index - 1]

    def IsOutsidePeriod(self, index:int = 0):
        return self.IsLowerLow(index) and self.IsHigherHigh(index)        

    def IsInsidePeriod(self, index:int = 0):        
        return self.IsHigherOrEqualLow(index) and self.IsLowerOrEqualHigh(index)        

    def next(self):
        # self.logdata()
        self.calc()
        
    @dump_args
    def notify_order(self, order):
        # import pdb; pdb.set_trace()
        print(str(order))

    @dump_args
    def notify_trade(self, trade):
        pass
    
    # @dump_args
    def notify_cashvalue(self, cash, value):
        pass
    
    # @dump_args
    def notify_fund(self, cash, value, fundvalue, shares):
        pass
    
    def notify_store(self, msg, *args, **kwargs):
        pass
        
def run(args=None):
    datapath = 'data/EURUSD_M1_2010.h5'
    df = pd.read_hdf(datapath)
    data = bt.feeds.PandasData(dataname=df['2010-06-01':'2010-06-30']) 
    
    tf = bt.TimeFrame.Minutes  
    n = 240
    cerebro = bt.Cerebro(stdstats=False)
    cerebro.resampledata(data, timeframe=tf, compression=n)
    cerebro.addstrategy(St, bar_size=dt.timedelta(minutes=n))
    cerebro.addobserver(OrderObserver) 
    result = cerebro.run(stdstats=True)
    cerebro.plot(iplot=False, style='candle', volume=False, barupfill=False, barup='black', bardown='black')
    
if __name__ == '__main__':
    print("hello")
    run()

hello
__main__.St.OnBuyStage3 ( self = <__main__.St object at 0x11555bb00> )
__main__.St.OnBuyStage3 ( self = <__main__.St object at 0x11555bb00> )
__main__.St.OnBuyStage3 ( self = <__main__.St object at 0x11555bb00> )
__main__.St.OnBuyStage3 ( self = <__main__.St object at 0x11555bb00> )
2010-06-11 16:00:00: (1.20760, 1.20970, 1.20550, 1.20930) <Patterns.Pattern5|Buy: 65>
__main__.St.notify_order ( self = <__main__.St object at 0x11555bb00>, order = <backtrader.order.BuyOrder object at 0x11556ad30> )
Ref: 1
OrdType: 0
OrdType: Buy
Status: 1
Status: Submitted
Size: 22000
Price: 1.2098
Price Limit: 1.2099
TrailAmount: None
TrailPercent: None
ExecType: 4
ExecType: StopLimit
CommInfo: None
End of Session: 733934.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
__main__.St.notify_order ( self = <__main__.St object at 0x11555bb00>, order = <backtrader.order.SellOrder object at 0x11556ada0> )
Ref: 2
OrdType: 1
OrdType: Sell
Status: 1
Status: Submitted
Size: -22000
Price: 1.2054
Pr

__main__.St.notify_order ( self = <__main__.St object at 0x11555bb00>, order = <backtrader.order.BuyOrder object at 0x11557b7b8> )
Ref: 16
OrdType: 0
OrdType: Buy
Status: 1
Status: Submitted
Size: 19000
Price: 1.2406000000000001
Price Limit: 1.2407000000000001
TrailAmount: None
TrailPercent: None
ExecType: 4
ExecType: StopLimit
CommInfo: None
End of Session: 733941.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
__main__.St.notify_order ( self = <__main__.St object at 0x11555bb00>, order = <backtrader.order.SellOrder object at 0x11557b828> )
Ref: 17
OrdType: 1
OrdType: Sell
Status: 1
Status: Submitted
Size: -19000
Price: 1.2356
Price Limit: None
TrailAmount: 0.0051000000000001044
TrailPercent: None
ExecType: 5
ExecType: StopTrail
CommInfo: None
End of Session: 733941.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
__main__.St.notify_order ( self = <__main__.St object at 0x11555bb00>, order = <backtrader.order.SellOrder object at 0x11557b898> )
Ref: 18
OrdType: 1

__main__.St.notify_order ( self = <__main__.St object at 0x11555bb00>, order = <backtrader.order.BuyOrder object at 0x115592d30> )
Ref: 37
OrdType: 0
OrdType: Buy
Status: 1
Status: Submitted
Size: 38000
Price: 1.2328000000000001
Price Limit: 1.2329
TrailAmount: None
TrailPercent: None
ExecType: 4
ExecType: StopLimit
CommInfo: None
End of Session: 733944.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
__main__.St.notify_order ( self = <__main__.St object at 0x11555bb00>, order = <backtrader.order.SellOrder object at 0x115592da0> )
Ref: 38
OrdType: 1
OrdType: Sell
Status: 1
Status: Submitted
Size: -38000
Price: 1.2303
Price Limit: None
TrailAmount: 0.0026000000000001577
TrailPercent: None
ExecType: 5
ExecType: StopTrail
CommInfo: None
End of Session: 733944.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
__main__.St.notify_order ( self = <__main__.St object at 0x11555bb00>, order = <backtrader.order.SellOrder object at 0x115592e10> )
Ref: 39
OrdType: 1
OrdType: Se

__main__.St.notify_order ( self = <__main__.St object at 0x11555bb00>, order = <backtrader.order.BuyOrder object at 0x11570d7b8> )
Ref: 52
OrdType: 0
OrdType: Buy
Status: 1
Status: Submitted
Size: 18000
Price: 1.2382
Price Limit: 1.2383
TrailAmount: None
TrailPercent: None
ExecType: 4
ExecType: StopLimit
CommInfo: None
End of Session: 733951.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
__main__.St.notify_order ( self = <__main__.St object at 0x11555bb00>, order = <backtrader.order.SellOrder object at 0x11570d828> )
Ref: 53
OrdType: 1
OrdType: Sell
Status: 1
Status: Submitted
Size: -18000
Price: 1.2328000000000001
Price Limit: None
TrailAmount: 0.005499999999999838
TrailPercent: None
ExecType: 5
ExecType: StopTrail
CommInfo: None
End of Session: 733951.9999999999
Info: AutoOrderedDict()
Broker: None
Alive: True
__main__.St.notify_order ( self = <__main__.St object at 0x11555bb00>, order = <backtrader.order.SellOrder object at 0x11570d898> )
Ref: 54
OrdType: 1
OrdType: Sel